In [7]:
import time
import os
import random
import string
import pytz
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy.engine.base import Engine
from sqlalchemy import create_engine
from schema import Base

In [11]:
cwd = os.getcwd()
print(cwd)
# reading csv files
status_df: pd.DataFrame = read_data(cwd+'/data/store_status.csv')
time_zone_df: pd.DataFrame = read_data(cwd+'/data/time_zones.csv')
business_hours_df: pd.DataFrame = read_data(cwd+'/data/menu_hours.csv')

# Ensure timestamp_utc is in datetime format
status_df['timestamp_utc'] = pd.to_datetime(status_df['timestamp_utc'], format='mixed')

# Fill missing timezones with 'America/Chicago'
time_zone_df['timezone_str'].fillna('America/Chicago', inplace=True)

# Merge status_df with time_zone_df on store_id
status_df = status_df.merge(time_zone_df, on='store_id', how='left')

# Fill missing timezone_str with 'America/Chicago'
status_df['timezone_str'].fillna('America/Chicago', inplace=True)

# Merge business_hours_df with time_zone_df on store_id
business_hours_df = business_hours_df.merge(time_zone_df, on='store_id', how='left')

# Fill missing timezone_str with 'America/Chicago'
business_hours_df['timezone_str'].fillna('America/Chicago', inplace=True)


/home/vishal/Codebase/VSCodeProjects/loopai-fastApiProject/database


/tmp/ipykernel_2515297/2822089566.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  time_zone_df['timezone_str'].fillna('America/Chicago', inplace=True)
/tmp/ipykernel_2515297/2822089566.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [13]:
def convert_to_utc(local_time_str, dayOfWeek, tz_str):
    local_tz = pytz.timezone(tz_str)
    # Create a datetime object for the start of the week
    base_time = datetime.strptime('2024-08-05', '%Y-%m-%d') + timedelta(days=dayOfWeek)
    # Add the local time to the base_time
    local_time = datetime.strptime(local_time_str, '%H:%M:%S').replace(
        year=base_time.year, month=base_time.month, day=base_time.day
    )
    local_time = local_tz.localize(local_time)
    return local_time.astimezone(pytz.utc)

# Convert start_time_local and end_time_local to UTC
business_hours_df['start_time_utc'] = business_hours_df.apply(
    lambda row: convert_to_utc(row['start_time_local'], row['day'], row['timezone_str']), axis=1
)

business_hours_df['end_time_utc'] = business_hours_df.apply(
    lambda row: convert_to_utc(row['end_time_local'], row['day'], row['timezone_str']), axis=1
)

In [16]:
def calculate_uptime_downtime(status_df, business_hours_df, now):
    results = []

    for store_id in status_df['store_id'].unique():
        store_status = status_df[status_df['store_id'] == store_id]
        store_hours = business_hours_df[business_hours_df['store_id'] == store_id]
        
        # Assume 24/7 if no business hours data
        if store_hours.empty:
            store_hours = pd.DataFrame([{
                'store_id': store_id, 'day': day, 'start_time_utc': now - timedelta(days=7),
                'end_time_utc': now + timedelta(days=7), 'timezone_str': 'America/Chicago'
            } for day in range(7)])
        
        # Initialize uptime and downtime counters
        uptime_last_hour = downtime_last_hour = 0
        uptime_last_day = downtime_last_day = 0
        uptime_last_week = downtime_last_week = 0
        
        intervals = [
            ('last_hour', now - timedelta(hours=1)),
            ('last_day', now - timedelta(days=1)),
            ('last_week', now - timedelta(weeks=1))
        ]
        
        for interval_name, interval_start in intervals:
            uptime = 0
            downtime = 0
            current_time = interval_start
            
            while current_time < now:
                is_business_hours = False
                for _, hours_row in store_hours.iterrows():
                    if hours_row['start_time_utc'] <= current_time < hours_row['end_time_utc']:
                        is_business_hours = True
                        break
                
                if is_business_hours:
                    status_row = store_status[
                        (store_status['timestamp_utc'] <= current_time)
                    ].sort_values(by='timestamp_utc').iloc[-1]
                    status = status_row['status'] if not status_row.empty else 'inactive'
                    if status == 'active':
                        uptime += 1
                    else:
                        downtime += 1
                
                current_time += timedelta(minutes=1)
            
            if interval_name == 'last_hour':
                uptime_last_hour = uptime
                downtime_last_hour = downtime
            elif interval_name == 'last_day':
                uptime_last_day = uptime / 60
                downtime_last_day = downtime / 60
            elif interval_name == 'last_week':
                uptime_last_week = uptime / 60
                downtime_last_week = downtime / 60
        
        results.append({
            'report_id': 'report1',
            'store_id': store_id,
            'uptime_last_hour': uptime_last_hour,
            'downtime_last_hour': downtime_last_hour,
            'uptime_last_day': uptime_last_day,
            'downtime_last_day': downtime_last_day,
            'uptime_last_week': uptime_last_week,
            'downtime_last_week': downtime_last_week
        })
    
    return pd.DataFrame(results)

# Calculate uptime and downtime
now = pd.Timestamp.utcnow()
report_df = calculate_uptime_downtime(status_df, business_hours_df, now)


# inserting data into the database.
start_time = time.time()
try:
    # connecting to SQLite.
    engine = get_connection()
    report_df.to_sql('reports', con=engine, chunksize=10000, if_exists='replace', index=False)
except Exception as insert_exception:
    print(f'Unable to populate tables, {repr(insert_exception)}')
else:
    end_time = time.time()
    execution_time = (end_time - start_time) / 60
    print(f'Successful in creating and populating the tables in {execution_time:.2f} minutes')


KeyboardInterrupt: 